In [328]:
import camelot
import pandas
import numpy as np

In [329]:
# tables = camelot.read_pdf("JBCAP.pdf", pages="80-140",flavor='stream')

In [330]:
print(tables[26])

<Table shape=(37, 14)>


In [331]:
df_main = pandas.DataFrame()

for table in tables:
    df_main = pandas.concat([df_main, table.df], ignore_index=True)

In [332]:
key_words = ['Direct Equity','Sell Date']

In [333]:
r1 = df_main[0].value_counts()['Sell Date']

In [334]:
for x in range(0,r1):
    start_label_index = df_main.index[df_main[0] == key_words[0]][0]
    end_label_index = df_main.index[df_main[0] == key_words[1]][0]
    start_pos = df_main.index.get_loc(start_label_index)
    end_pos = df_main.index.get_loc(end_label_index)

    end_pos = end_pos+3
    indices_to_drop = df_main.iloc[start_pos:end_pos].index

    df_main.drop(indices_to_drop, inplace=True)

In [335]:
words = ["Vimuras Family Private Trust", "\\*Refer Disclaimer at the end of the report.","Equity","Taxable Gain & Loss Statement"]
pattern = "|".join(words)

Check each row if it has the above if so then delete the row, proboly need to loop

In [336]:
df_main = df_main[~df_main[0].str.contains(pattern, na=False, case=False)]
df_main = df_main[~df_main[5].str.contains(pattern, na=False, case=False)]

In [337]:
df_main.reset_index(inplace=True,drop=True)

In [338]:
def is_comp(val):
    if not isinstance(val, str):
        return False

    # 2. Strip whitespace from the string. This is the key change.
    #    "   Company A   " -> "Company A"
    #    "       "         -> ""
    cleaned_val = val.strip()

    # 3. Check if the cleaned string is empty. If it is, it's not a company name.
    #    This now correctly handles strings that were originally just whitespace.
    if not cleaned_val:
        return False

    # 4. Now that we know we have a non-empty, cleaned string, apply the final rules.
    if cleaned_val.lower() == 'total' or cleaned_val[0].isdigit():
        return False

    # 5. If it passed all the checks, it's a company name.
    return True


In [339]:
mask1 = df_main[0].apply(is_comp)
mask2 = df_main[1].apply(is_comp)


In [340]:
mask1.value_counts()

0
False    2492
True      201
Name: count, dtype: int64

In [341]:
combine_mask = mask1 | mask2

In [342]:
company_names = pandas.Series(np.where(mask1, df_main[0], df_main[1]))

In [343]:
company_col = company_names.where(combine_mask).ffill()

In [344]:
df_main = df_main[~combine_mask]

In [345]:
df_main.insert(0, "Company", company_col)

In [347]:
df_main.columns = ["Company", "Sell Date", "Quantity", "Sell Rate", "Total Sale Value", "Purchase Date", "Purchase Rate","Actual Cost", "FMV as on 31-01-2018/Indexed Rate", "Applicable Rate", "Effective Cost", "Days Held", "Short Term", "Long Term", "Effective LT"]